# Setup


In [3]:
import pandas as pd
import numpy as np
from glob import glob
from re import match
from matplotlib import pyplot as plt
import seaborn as sns
from sys import stdout
from collections import defaultdict
from tqdm import tqdm
from itertools import product
from json import dumps, loads

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import (
    MAPE, MeanAbsolutePercentageError, MSE
)
from tensorflow.keras.callbacks import (
    Callback, LearningRateScheduler, ModelCheckpoint
)
from tensorflow.keras.regularizers import L1


In [4]:
def scheduler1(epoch, lr): 
    """
Learning rate scheduler that alternates between higher and lower rates.\
"""
    return 0.0001 if epoch % 2 == 1 else 0.001
    
def scheduler2(epoch, lr):
    """\
Learning rate scheduler that alternates between higher and \
lower rates for half of the training and uses the lower rate afterwards.\
"""
    return 0.0001 if ((epoch % 2 == 1) or (epoch > epochs/2)) else 0.001
    
def scheduler3(epoch, lr):
    """Learning rate scheduler that only uses the lower rate."""
    return 0.0001

def scheduler4(epoch, lr):
    """\
Learning rate scheduler that alternates between higher and \
lower rates until the final 100 epochs, when only the lower rate is used.\
"""
    return 0.0001 if ((epoch % 2 == 1) or (epoch > epochs-100)) else 0.001

class ProgressBar(Callback):
    """\
A progress bar displays while fitting models \
showing the relative progess and estimated time to competion.\
"""

    def on_train_begin(self, logs=None):
        self.epochs = self.__dict__['params']['epochs']
        self.steps = self.__dict__['params']['steps'] 
        # Calculate the number of steps to completion.
        self.total_steps = self.steps * self.epochs 
        # Generate progress bar.
        self.progress_bar = tqdm(
            desc='Training', total=self.total_steps, unit=' step', 
            smoothing=0, file=stdout, colour='green', position=0, 
            ncols=100, unit_scale=1/self.steps,
            bar_format='{l_bar}{bar}| {n:.0f}/{total:.0f} epochs \
completed [{elapsed}<{remaining}, {rate_fmt}]',
        )
    def on_batch_end(self, batch, logs=None):
        # Update the progress bar by one step after every batch.
        self.progress_bar.update(1)

    def on_train_end(self, logs=None):
        # Close the bar when training is finished.
        self.progress_bar.close()

class SaveTrainTime(Callback):
    '''\
Record the training session metadata when a model completes fitting.\
'''
    
    def on_train_begin(self, logs=None):
        self.start_time = datetime.now()

    def on_train_end(self, logs=None):
        # Load and format the CSV of training times.
        ttimes = pd.read_csv('models/train_times.csv')
        ttimes.train_time = ttimes.train_time.astype('timedelta64[ns]')
        params = self.__dict__['params']
        # Record the current session and save the results.
        ttimes = pd.concat([
            ttimes,
            pd.DataFrame([[
                self.model.name, 
                self.start_time, 
                datetime.now() - self.start_time, 
                params['epochs'], 
                params['steps'],
            ]], columns=ttimes.columns) 
        ])
        
        ttimes.to_csv('models/train_times.csv', index=False)

def get_MLP(X_train, n_hidden=1, univariate=False, model_prefix=''):
    """Build a multilayer perception. Return the uncompiled model.

    Args:
        X_train (tf.Tensor): Input training data.
        n_hidden (int): Number of hidden layer in neural network.
        univariate (Bool): Whether or not the model to be \
produced is univariate.
        model_prefix (string): Text to be prepended to the model name.

    Returns:
        model_kwargs (dict): A dictionary including the uncompiled model\
 to pass to train_model() as kwargs.
    """
    tf.keras.utils.set_random_seed(1)

    # Define the input layer.
    main_input = Input(shape=tuple(X_train[0].shape), name="input")
    previous_layer = main_input

    # Add as many hidden layers as are specified, 
    # alternating between linear and relu activation functions.
    for i in range(n_hidden):
        if i % 2 == 1:
            previous_layer = Dense(
                32, name=f"linear_{i+1}", activation='linear'
            )(previous_layer)
        else:
            previous_layer = Dense(
                32, name=f"relu_{i+1}", activation='relu'
            )(previous_layer)

    # Create the linear output layer.
    main_output = Dense(2, name=f"output")(previous_layer)

    # Name the model.
    model_prefix = model_prefix + '_' if model_prefix else model_prefix
    name = f"""{model_prefix}MLP_{
        'shallow' if n_hidden <= 1 else 'deep'
    }_{
        'univ' if univariate else 'multiv'
    }"""

    # Return model and additional information 
    # for the train_model() function.
    return {
        'model': Model(
            inputs=main_input, outputs=main_output, name=name
        ),
        'univariate': univariate,
        'X_train': X_train,
    }

def get_LSTM(X_train, n_hidden=1, univariate=False, model_prefix=''):
    """Build a bidirectional LSTM. Return the uncompiled model.

    Args:
        X_train (tf.Tensor): Input training data.
        n_hidden (int): Number of hidden layer in neural network.
        univariate (Bool): Whether or not the model will be univariate.
        model_prefix (string): Text to be prepended to the model name.

    Returns:
        model_kwargs (dict): A dictionary including the uncompiled model\
 to pass to train_model() as kwargs.
    """
    tf.keras.utils.set_random_seed(1)
    
    # Define the input layer.
    main_input = Input(shape=(X_train[0].shape[0],1), name="input")
    previous_layer = main_input
    # Add as many hidden layers as are specified, starting with the 
    # widest. The width is defined by powers of 2.
    for i in range(n_hidden-1):
        previous_layer = Bidirectional(
            LSTM(2**(4+n_hidden-i), return_sequences=True), 
            name=f"BD_{i+1}"
        )(previous_layer)
    previous_layer = Bidirectional(
        LSTM(2**5), name=f"BD_{n_hidden}"
    )(previous_layer)

    # Create the linear output layer.
    main_output = Dense(2, name=f"output")(previous_layer)

    # Name the model.
    model_prefix = model_prefix + '_' if model_prefix else model_prefix
    name = f"""{model_prefix}LSTM_{
        'shallow' if n_hidden <= 1 else 'deep'
    }_{
        'univ' if univariate else 'multiv'
    }"""
    
    # Return model and other information for the train_model() function.
    return {
        'X_train': X_train,
        'model': Model(
            inputs=main_input, outputs=main_output, name=name
        ), 'univariate': univariate,
    }

def _compile_model(model, opt_params={}):
    """Compile an uncompiled Tensorflow model. Return the compiled model.

    Args:
        model (keras.src.models.functional.Functional): \
The uncompiled model.
        opt_params (dict): A kwargs dictionary used with the optimiser.

    Returns:
        model (keras.src.models.functional.Functional): \
The compiled model.
    """
    
    model.compile(
        optimizer=Adam(**opt_params), 
        loss=MeanAbsolutePercentageError(), 
        metrics=[MSE]
    )
    return model

# Define the optimiser hyperparameters of the BD-LSTMs. This is declared 
# globally, as it is used in both train_model() and load_lstm().
lstm_opt_params = {'weight_decay': 1e-05, 'beta_1': 0.8}

def train_model(
    y_train, X_val, y_val, X_train, model, 
    univariate=False, epochs=2000, opt_params={}, fit_params={}
):
    """Train an uncompiled Tensorflow model. \
Return the trained model and its training history.

    Args:
        y_train (tf.Tensor): Target training data.
        X_val (tf.Tensor): Input validation data.
        y_val (tf.Tensor): Target validation data.
        X_train (tf.Tensor): Input training data.
        model (keras.src.models.functional.Functional): \
The uncompiled model.
        univariate (bool): True if model is univariate and False \
if multivariate.
        epochs (int): Number of training epochs.
        opt_params (dict): A kwargs dictionary used with the model's \
optimiser.
        fit_params (dict): A kwargs dictionary used with the model's \
fit() method.

    Returns:
        model (keras.src.models.functional.Functional): The trained model.
        history (pd.DataFrame): The record of training and validation \
loss and metric values at successive epochs.
    """
    # Define the hyperparameters and checkpoint callback for the model. 
    if 'MLP' in model.name:
        opt_params = opt_params if opt_params \
            else {'weight_decay': 1e-05, 'beta_1': 0.95}
        fit_params = fit_params if fit_params \
            else {'batch_size': 2**8}
        # The model checkpoint will save the best model, 
        # based on validation loss, from the fitting process.
        model_checkpoints = ModelCheckpoint(
            f"models/{model.name}.keras", save_best_only=True
        )
    elif 'LSTM' in model.name:
        opt_params = opt_params if opt_params else lstm_opt_params
        fit_params = fit_params if fit_params else {'batch_size': 2**8}
        # The model checkpoint will save the best model, 
        # based on validation loss, during the fitting process.
        # A bug in Tensorflow has meant that only the weights of 
        # BD-LSTMs can be loaded, not the whole model in a KERAS file. 
        model_checkpoints = ModelCheckpoint(
            f"models/{model.name}.weights.h5", 
            save_best_only=True, save_weights_only=True
        )

    model = _compile_model(model, opt_params=opt_params)

    # Fit the model on the training day 
    # while testing, after each epoch, on the validation data.
    history = model.fit(
        X_train, y_train, verbose=0, shuffle=False, epochs=epochs,
            validation_data=(X_val, y_val), callbacks=[
            model_checkpoints, # Save the model.
            # Alternate the learning rate.
            LearningRateScheduler(scheduler1), 
            ProgressBar(), # Show the progress bar.
            SaveTrainTime(), # Save the fitting metadata.
        ], **fit_params
    )
    # Record the training history locally.
    history = pd.DataFrame(history.history)
    history.to_csv(
        f'models/{model.name} history.csv', 
        index=False, lineterminator='\n'
    )
    
    return model, history

def load_lstm(name):
    """Take the name of a model and return the most trained model \
stored locally. Note that this method will become redundant once a \
bug in Tensorflow stopping KERAS files of these models from being \
loaded is fixed."""
    # Select the right data for this model.
    if 'univ' in name:
        _X_train, _y_train, _X_test, _y_test = \
            X_train_uv, y_train_full, X_test_uv, y_test
        univariate = True
    else:
        _X_train, _y_train, _X_test, _y_test = \
            X_train_full, y_train_full, X_test, y_test
        univariate = False
    # Define the depth of the architecture.
    if 'deep' in name:
        n_hidden = 2
    else:
        n_hidden = 1

    # Get the prefix of the most trained model stored locally.
    prefix = name.split('_LSTM')[0]
    # Build the model.
    lstm_data = get_LSTM(
        _X_train, n_hidden, univariate=univariate, model_prefix=prefix
    )
    model = _compile_model(lstm_data['model'])

    # Load the saved weights.
    model.load_weights(f"models/{name}.weights.h5")

    # Re-apply the hyperparameters used in the saved model.
    for k, v in lstm_opt_params.items():
        exec(f'model.optimizer.{k} = {v}')

    return model

def flatten_params(param_grid):
    """Take a paramater grid and return a flattened list of each \
possible combination of the parameters."""
    return [
        dict(zip(param_grid.keys(), e)) 
        for e in product(*param_grid.values())
    ]

def GridSearchCV(estimator, param_grid, epochs):
    """Take a callable model generator and a parameter grid and \
return the best possible set of parameters given the specified epochs.

    Args:
        estimator (function): A function returning an uncompiled model.
        param_grid (dict): A dictionary of lists of hyperparameters \
specifiying the hyperparameter space to be searched.
        epochs (int): Number of training epochs.

    Returns:
        dict: The hyperparameters used to train the best model.
    """
    est_params = {} # Parameters for building the model.
    opt_params = {} # Parameters of the optimiser.
    params = {} # Parameters of Model.fit method.
    # Split the parameter grid into separate dictionaries.
    for k, v in param_grid.items():
        if (est_lab := 'estimator__') in k:
            est_params.update({k.replace(est_lab, ''): v})
        elif (opt_lab := 'optimizer__') in k:
            opt_params.update({k.replace(opt_lab, ''): v})
        else:
            params.update({k: v})

    # Expand the grid into each combination to be tested.
    param_grid_list = flatten_params({
        'est_params': flatten_params(est_params), 
        'opt_params': flatten_params(opt_params), 
        'params': flatten_params(params),
    })
    n_iter = len(param_grid_list)
    results = defaultdict(list)

    # Test each combination of parameters. 
    # Represent the process with a progress bar.
    for i in tqdm(
        range(n_iter), desc='Searching hyperparameter space', 
        file=stdout, colour='green'
    ):
        param_grid = param_grid_list[i]
        # For each train/validation set...
        for j in range(len(X_train_sets)):
            # Load the train/validation data.
            X_train, y_train, X_val, y_val = \
                X_train_sets[j], y_train_sets[j], \
                X_val_sets[j], y_val_sets[j]

            # Get the current test parameters and run the model.
            est_params = param_grid['est_params']
            opt_params = param_grid['opt_params']
            params = param_grid['params']

            model, _, _ = estimator(X_train, **est_params).values()
            
            model.compile(
                optimizer=Adam(**opt_params), 
                loss=MeanAbsolutePercentageError(), 
            )
            history = model.fit(
                X_train, y_train, verbose=0, shuffle=False, 
                epochs=epochs, validation_data=(X_val, y_val), 
                callbacks=[LearningRateScheduler(scheduler1)], 
                **params
            )
            # Save the results.
            results[i].append(model.evaluate(X_val, y_val, verbose=0))

    # Return the set of parameters that have 
    # the lowest mean validation loss.
    return param_grid_list[pd.Series(results).apply(np.mean).argmin()]

def extend_training(model):
    '''Rename a previous model so that it can be further trained \
without overwriting any records.'''
    model.name = f"Extended_{model.name}"
    return model

def get_latest_models(name=None):
    '''Return information on the most trained iterations of over model. \
If a specific model is named, only return the name of the most trained \
iteration.'''
    # List each model stored locally.
    all_models = pd.DataFrame([
        e.replace('\\', '/') for e in glob('models/*') 
        if match(r'.*\.(?:keras|h5)$', e)
    ], columns=['file'])
    # Extract the model name, the iteration name, the count of training 
    # sessions, and whether it is the latest iteration.
    all_models['model'] = all_models.file.str\
        .extract(r'models/.*Final_(.*?)\..*$')
    all_models['model_name'] = all_models.file.str\
        .extract(r'models/(.*Final_.*?)\..*$')
    all_models['extensions'] = all_models.file\
        .apply(lambda e: e.count('Extended'))
    all_models['latest'] = all_models.groupby('model')\
        .extensions.transform('max')
    # Filter out older iterations.
    latest = all_models.loc[
        all_models.extensions == all_models.latest, 
        ['model', 'model_name', 'file']
    ]
    
    if name:
        latest = latest.loc[latest.model == name, 'model_name']\
            .squeeze()
    
    return latest



# Data preparation


In [5]:
# Load and prepare the modelling data.
df = pd.read_csv(
    'data/modelling_data.csv', parse_dates=['DATETIME'], 
    date_format='%Y-%m-%d %H:%M:%S'
)
df = df.set_index('DATETIME').dropna().sort_index()

# Split the dataframe into X and y data.
y_cols = ['h1_TOTALDEMAND', 'h24_TOTALDEMAND']
X_df, y_df = df.drop(columns=y_cols), df[y_cols]

# Scale the data.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)

# Split the data into train/validation and test sets.
X_y = [X_train_full, X_test, y_train_full, y_test] = train_test_split(
    X_scaled, y_df, test_size=0.2, shuffle=False
)

# Convert the data to tensors.
X_y = [tf.convert_to_tensor(d) for d in X_y]
[X_train_full, X_test, y_train_full, y_test] = X_y

# Create a separate univariate data set.
uv_cols = np.array([
    bool(match(r'TOTALDEMAND|TM\d+', e)) for e in X_df.columns
])
X_train_uv = tf.convert_to_tensor(X_train_full.numpy()[:,uv_cols])
X_test_uv = tf.convert_to_tensor(X_test.numpy()[:,uv_cols])

# Generate the time-series cross-validation train and validation sets. 
train_val_I = TimeSeriesSplit(n_splits=5).split(X_train_full)
X_train_sets, y_train_sets, X_val_sets, y_val_sets = [], [], [], []

for train, val in train_val_I:
    # Print the traind and validation set lengths.
    print(train.shape, val.shape) 
    val_start, val_end = val.min(), val.max()+1
    X_train_sets.append(X_train_full[:val_start])
    y_train_sets.append(y_train_full[:val_start])
    X_val_sets.append(X_train_full[val_start: val_end])
    y_val_sets.append(y_train_full[val_start: val_end])

X_train_sets


(104758,) (104755,)
(209513,) (104755,)
(314268,) (104755,)
(419023,) (104755,)
(523778,) (104755,)


[<tf.Tensor: shape=(104758, 70), dtype=float64, numpy=
 array([[ 0.9132249 ,  0.9131219 , -1.58047303, ...,  0.87825308,
          1.00316397,  1.09089092],
        [ 0.09708733,  0.86111884, -1.58047303, ...,  0.04669082,
          0.0874565 ,  0.1407623 ],
        [-0.15027044,  0.01142544, -1.58047303, ..., -0.08985632,
         -0.03193541,  0.08894003],
        ...,
        [-1.47230972,  0.30641948, -1.27130153, ..., -1.38906997,
         -1.3868722 , -1.36763819],
        [ 1.74346793, -0.57763261, -1.27130153, ...,  1.67796777,
          1.66124456,  1.67802582],
        [ 0.4047031 , -0.02989103, -1.27130153, ...,  0.4411795 ,
          0.45533762,  0.47632769]])>,
 <tf.Tensor: shape=(209513, 70), dtype=float64, numpy=
 array([[ 0.9132249 ,  0.9131219 , -1.58047303, ...,  0.87825308,
          1.00316397,  1.09089092],
        [ 0.09708733,  0.86111884, -1.58047303, ...,  0.04669082,
          0.0874565 ,  0.1407623 ],
        [-0.15027044,  0.01142544, -1.58047303, ..., -0.08

# Hyperparemeter tuning

The code below is illustrative of how the gridsearch results were found. The hyperparameter values printed under each cell became the defaults in the `get_MLP` and `get_LSTM` functions after this experiment.


In [16]:
# param_grid={
#     'batch_size': [2**8, 2**9, 2**10],
#     'optimizer__weight_decay': [1e-3, 1e-4, 1e-5],
#     'optimizer__beta_1': [.8, .9, .95],
# }

# results = GridSearchCV(get_MLP, param_grid, epochs=300)

# with open('Results/MLP gridsearch results.json', 'w') as f:
#     f.write(dumps(results, indent=4))

with open('Results/MLP gridsearch results.json', 'r') as f:
    results = loads(f.read())
    
results


{'opt_params': {'weight_decay': 1e-05, 'beta_1': 0.95},
 'params': {'batch_size': 256}}

In [17]:
# results = GridSearchCV(get_LSTM, param_grid, epochs=10)

# with open('Results/BD-LSTM gridsearch results.json', 'w') as f:
#     f.write(dumps(results, indent=4))

with open('Results/BD-LSTM gridsearch results.json', 'r') as f:
    results = loads(f.read())
    
results


{'opt_params': {'weight_decay': 1e-05, 'beta_1': 0.8},
 'params': {'batch_size': 256}}

# Modelling

The code below is also somewhat illustrative. As the models were trained in iterations and asynchronously, the script that generated them never existed in one file. As such, I have provided the code to recreate or load the models we ultimately generated. The numbers of epochs listed are the total epochs we used.


## Shallow multivariate MLP (M1)

In [8]:
# m1, m1_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=10_000, **get_MLP(X_train_full, model_prefix='Final'),
# )

# m1.evaluate(X_test, y_test)

# Load the model generated by the code above. 
m1 = load_model(
    f"models/{get_latest_models('MLP_shallow_multiv')}.keras", 
    safe_mode=True
)


## Deep multivariate MLP (M2)

In [ ]:
# m2, m2_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=4000, **get_MLP(X_train_full, 10, model_prefix='Final'),
# )

# m2.evaluate(X_test, y_test)

# Load the model generated by the code above. 
m2 = load_model(
    f"models/{get_latest_models('MLP_deep_multiv')}.keras", 
    safe_mode=True
)


## Shallow multivariate BD-LSTM (M3)

In [ ]:
# m3, m3_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=511, **get_LSTM(X_train_full, model_prefix='Final'),
# )

# m3.evaluate(X_test, y_test)

# Load the model generated by the code above. 
m3 = load_lstm(get_latest_models('LSTM_shallow_multiv'))


## Deep multivariate BD-LSTM (M4)


In [1]:
# m4, m4_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=189, **get_LSTM(X_train_full, 2, model_prefix='Final'),
# )

# m4.evaluate(X_test, y_test)

# Load the model generated by the code above. 
m4 = load_lstm(get_latest_models('LSTM_deep_multiv'))


## Shallow univariate MLP (M5)


In [208]:
# m5, m5_hist = train_model(
#    y_train_full, X_test_uv, y_test, epochs=10_500, 
#    **get_MLP(X_train_uv, univariate=True, model_prefix='Final'),
# )

# m5.evaluate(X_test_uv, y_test)

# Load the model generated by the code above. 
m5 = load_model(
    f"models/{get_latest_models('MLP_shallow_univ')}.keras", 
    safe_mode=True
)


## Deep univariate MLP (M6)


In [10]:
# m6, m6_hist = train_model(
#     y_train_full, X_test_uv, y_test, epochs=3950, 
#     **get_MLP(X_train_uv, 10, univariate=True, model_prefix='Final'),
# )

# m6.evaluate(X_test_uv, y_test)

# Load the model generated by the code above. 
m6 = load_model(
    f"models/{get_latest_models('MLP_deep_univ')}.keras", 
    safe_mode=True
)


## Shallow univariate BD-LSTM (M7)


In [11]:
# m7, m7_hist = train_model(
#     y_train_full, X_test_uv, y_test, epochs=315, 
#     **get_LSTM(X_train_uv, univariate=True, model_prefix='Final'),
# )

# m7.evaluate(X_test_uv, y_test)

# Load the model generated by the code above. 
m7 = load_lstm(get_latest_models('LSTM_shallow_univ'))


## Deep univariate BD-LSTM (M8)


In [12]:
# m8, m8_hist = train_model(
#     y_train_full, X_test_uv, y_test, epochs=150, 
#     **get_LSTM(X_train_uv, 2, univariate=True, model_prefix='Final'),
# )

# m8.evaluate(X_test_uv, y_test)

# Load the model generated by the code above. 
m8 = load_lstm(get_latest_models('LSTM_deep_univ')) 


# Model results


In [8]:
# Generate the latest version of each model and store it in a dataframe.
model_files = get_latest_models()
models = dict()


for m, n, f in model_files.values:
    model = load_lstm(n) if 'LSTM' in m else load_model(f)
    models[m] = model
        
models_df = pd.DataFrame(models.items(), columns=['model', 'object'])

models_df


,model,object
0,MLP_shallow_multiv,"<Functional name=Final_MLP_shallow_multiv, bui..."
1,LSTM_shallow_multiv,<Functional name=Extended_Extended_Final_LSTM_...
2,LSTM_shallow_univ,<Functional name=Extended_Final_LSTM_shallow_u...
3,LSTM_deep_multiv,<Functional name=Extended_Extended_Final_LSTM_...
4,MLP_deep_multiv,"<Functional name=Final_MLP_deep_multiv, built=..."
5,LSTM_deep_univ,<Functional name=Extended_Final_LSTM_deep_univ...
6,MLP_shallow_univ,"<Functional name=Final_MLP_shallow_univ, built..."
7,MLP_deep_univ,"<Functional name=Final_MLP_deep_univ, built=True>"


In [11]:
# List the requisite X data for evaluating the models.
models_df['X_test'] = models_df.model\
    .apply(lambda e: 'X_test_uv' if 'univ' in e else 'X_test')

# Calculate the test MAPE and MSE.
models_df[['test_MAPE', 'test_MSE']] = models_df.apply(lambda d: eval(
    f"d['object'].evaluate({d['X_test']}, y_test)"
), axis=1).apply(pd.Series)

# Convert MSE to RMSE.
models_df['test_RMSE'] = models_df.test_MSE**.5

# Create the predictions for each model.
models_df['y_pred_24H'] = models_df.apply(lambda d: eval(
    f"d['object'].predict({d['X_test']})"
), axis=1).apply(lambda d: [d[:,i].reshape(1, -1)[0] for i in range(2)])

models_df['y_pred_1H'] = models_df.y_pred_24H.str.get(0)
models_df['y_pred_24H'] = models_df.y_pred_24H.str.get(1)

# Use the predictions to record the MAPE and RMSE 
# for each prediction horizon (i.e. 1 or 24 hours).
for i, h in enumerate([1, 24]):
    y_test_H = y_test[:,i]
    models_df[f'test_MAPE_{h}H'] = models_df[f'y_pred_{h}H']\
        .apply(lambda d: float(MAPE(y_test_H, d)))
    models_df[f'test_RMSE_{h}H'] = models_df[f'y_pred_{h}H']\
        .apply(lambda d: float(MSE(y_test_H, d)**.5))

# Save the data to be reviewed in the results.
models_df[[
    'model', 'test_MAPE', 'test_RMSE', 'test_MAPE_1H', 
    'test_RMSE_1H', 'test_MAPE_24H', 'test_RMSE_24H'
]].to_csv('Results/model test results.csv', index=False)


4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 148us/step - loss: 4.5896 - mean_squared_error: 110105.2500
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 6.2300 - mean_squared_error: 229217.9844
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 6.0058 - mean_squared_error: 242343.8125
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 48s 10ms/step - loss: 5.5444 - mean_squared_error: 239986.5938
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 182us/step - loss: 5.5199 - mean_squared_error: 163765.6250
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - loss: 6.9838 - mean_squared_error: 472648.8750
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 144us/step - loss: 6.0822 - mean_squared_error: 172376.6719
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 186us/step - loss: 7.9082 - mean_squared_error: 230104.8281
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 166us/step
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 50s 10ms/step
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 205us/step
4911/4911 ━━━━━━━━━━━━━━━━━━━